In [ ]:
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import random
import numpy as np

n=0
N=10000
star=[]
while n<=N:
    rt=np.array([random.random(),random.random()])
    plt.scatter(rt[0],rt[1])
    n+=1
    
